In [1]:
%matplotlib inline
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython import display

# Motion planning with Rapidly-exploring Random Tree (RRT) 

Motion planning is the task to find a feasible and collision-free path from a given pose to a goal pose. Ideally, a plan should respect the robot’s kinematics and dynamics constraints. Rapidly-exploring Random Tree (RRT) is a single-query, sampling-based motion planning algorithm that works by building a tree graph, where the nodes represent states, and the edges represent what states are reachable from the others. RRT assumes that the map of the environment and the kinematics/dynamics of the robot are known. It also assumes that we have the ability to check for collision, as well as the ability to compute distance between two states. The algorithm is shown below.

<figure>
  <div style="text-align:center;">
  <img src="assets/04/rrt_algo.png">
  <figcaption>Source: Sertac Karaman and Emilio Frazzoli. Sampling-based Algorithms for Optimal Motion Planning. The International Journal of Robotics Research, 2011.</figcaption>
  </div>
</figure>

Given a goal state $x_{goal}$ and an initial state $x_{init}$, we first start with a graph with only $x_init$ as the only node. We then sample a random state $x_{rand}$, and find a node from the existing graph that is nearest to $x_{rand}$; we denote this nearest state as $x_{nearest}$. We then attempt to steer from $x_{nearest}$ towards $x_{rand}$ and get a new point $x_{new}$ that is between these two states (we will see how later in the example). Ideally, the steer function must adhere to the kinematics/dynamics of the robot. We then check if there are obstacles between $x_{nearest}$ and $x_{new}$. If there are no obstacles, we proceed to grow the graph by connecting $x_{nearest}$ and $x_{new}$. We then repeat this processes for $n$ steps, or until we find a path from $x_{init}$ to $x_{goal}$. Once done, we can obtain the path by going backwards from $x_{goal}$ to $x_{init}$. 

Although the path may not be optimal, RRT is guaranteed to find a feasible path if it exists. Let's take a look at how to implement RRT by doing an example.

**EXAMPLE: finding a feasible path to a goal position with RRT**

Suppose we are given a map with known obstacles where the robot starts at (0,0) and the goal is located at (4,4). Find a feasible path that will take the robot from its current location to the goal.

Note: for simplicity, let's assume our robot is able to reach a target position by rotating around (so that it points towards the position it wants to reach), and move forward until it reaches the target position.

In [ ]:
def plot_map(obstacles, cur_pos, goal_pos):
    """
    Function to visualize the map, initial position of the robot,
    and the goal position.
    """
    for x,y in obstacles:
        plt.gca().add_patch(patches.Circle((x, y), radius=r))
    plt.gca().add_patch(patches.Circle(cur_pos, radius=0.2, color='orange'))
    plt.gca().add_patch(patches.Circle(goal_pos, radius=0.2, color='green'))
    x_min, x_max = -5, 5
    y_min, y_max = -5, 5
    plt.xlim([x_min, x_max])
    plt.ylim([y_min, y_max])
    plt.gca().set_aspect('equal', adjustable='box')

In [ ]:
obstacles = [(1,1), (1,3), (-1,2),
             (-1,-2), (-4,0), (-3,4),
             (-3,-2), (4,-4), (3,2),
             (2,-2), (0,-4), (-3,2)] # Assume we know where the obstacles are
r = 0.4 # radius of obstacles
cur_pos = (0,0)
goal_pos = (4,4)
plot_map(obstacles, cur_pos, goal_pos)

In [ ]:
class Node:
    def __init__(self, x, y, parent=None):
        self.x = x # x-coordinate
        self.y = y # y-coordinate
        self.parent = parent # each node has exactly one parent

        
def get_node_dist(node_a, node_b):
    """
    Compute Euclidean distance between two nodes.
    Input:
        - node_a: node a (Node type)
        - node_b: node a (Node type)
    Return:
        - Euclidean distance between node_a and node_b
    """
    return np.sqrt((node_a.x - node_b.x)**2 + (node_a.y - node_b.y)**2)
    
        
def sample_free(goal_x, goal_y, goal_sample_rate=0.5, x_min=-5, x_max=5, y_min=-5, y_max=5):
    """
    With probability=goal_sample_rate, sample goal position,
    else, sample random point within the map.
    Input:
        - goal_x: x-coordinate of the goal position (float)
        - goal_y: y-coordinate of the goal position (float)
        - goal_sample_rate: probability of sampling goal position
        - x_min, x_max, y_min, y_max: min/max value of x and y that defines the map
    Return:
        - x_rand: a node (Node type)
    """
    if np.random.random() > goal_sample_rate:
        x_pos = np.random.uniform(x_min, x_max)
        y_pos = np.random.uniform(y_min, y_max)
        x_rand = Node(x_pos, y_pos)
    else:
        x_rand = Node(goal_x, goal_y)
    return x_rand


def nearest(x_rand, nodes):
    """
    Given x_rand, find the nearest node in nodes and return the index
    of the nearest node.
    Input:
        - x_rand: a node (Node type)
        - nodes: a list of nodes
    Return:
        - nearest_id: index of the nearest node in nodes (int)
    """
    distances = []
    for i in range(len(nodes)):
        node = nodes[i]
        dist = get_node_dist(x_rand, node)
        distances.append(dist)
    nearest_idx = np.argmin(distances)
    return nearest_idx
    
    
def steer(x_nearest, x_rand, branch_length = 0.2):
    """
    Get a new node that is located between x_nearest and x_rand,
    and branch_length away from x_nearest.
    NOTE: Alternatively, we can use inverse kinematics (IK) and simulate driving 
    with actions calculated using IK so all the paths in the graph are kinematically
    feasible.
    Input:
        - x_nearest: closest node from x_rand which we get using nearest() (Node type)
        - x_rand: node sampled using sample_free() (Node type)
        - branch_length: maximum branchin length from x_nearest (float)
    Return:
        - new_node: a new node located between x_nearest and x_rand (Node type)
    """
    nearest_to_rand = np.array([x_rand.x - x_nearest.x,
                                x_rand.y - x_nearest.y]) # vector from x_nearest to x_rand
    unit_nearest_to_rand = nearest_to_rand / np.linalg.norm(nearest_to_rand) # unit vector from x_nearest to x_rand
    new_node_position = np.array([x_nearest.x, x_nearest.y]) + unit_nearest_to_rand * branch_length
    new_node = Node(new_node_position[0], new_node_position[1])
    return new_node
   
    
def obstacle_free(x_nearest, x_new, obstacles, distance_threshold=0.6, num_nodes_in_between=5):
    """
    Check if there is an obstacle between x_nearest and x_new.
    Input:
        - x_nearest: closest node from x_rand which we get using nearest() (Node type)
        - x_new: a new node located between x_nearest and x_rand from running steer() (Node type)
        - obstacles: list of obstacles
        - distance_threshold: distance threshold to obstacles -> not safe below this value (float)
        - num_nodes_in_between: number of nodes between x_nearest and x_new
    Return:
        - obstacle_free: a flag indicating if there is an obstacle between x_nearest and x_new (bool)
    """
    obstacle_free = True
    
    # First, populate nodes in between x_nearest and x_new
    nodes_in_between = []
    nearest_to_new = np.array([x_new.x - x_nearest.x,
                               x_new.y - x_nearest.y]) # vector from x_nearest to x_new
    unit_nearest_to_new = nearest_to_new / np.linalg.norm(nearest_to_new)
    spacing_between_nodes = get_node_dist(x_nearest, x_new) / float(num_nodes_in_between)
    for i in range(1, num_nodes_in_between + 1):
        node_in_between = np.array([x_nearest.x, x_nearest.y]) + unit_nearest_to_new * i * spacing_between_nodes
        node_in_between = Node(node_in_between[0], node_in_between[1])
        nodes_in_between.append(node_in_between)
    
    # Check if any of the nodes in nodes_in_between is in the way of an obstacle
    for node_in_between in nodes_in_between:
        for obs_x, obs_y in obstacles:
            dist = np.sqrt((node_in_between.x - obs_x)**2 + (node_in_between.y - obs_y)**2)
            if dist <= distance_threshold:
                obstacle_free = False
                return obstacle_free
                
    return obstacle_free


def rrt(x_init, x_goal, obstacles, success_distance_threshold=0.1):
    """
    Run RRT until we get a node that is close to the goal node.
    Input:
        - x_init: initial node (Node type)
        - x_goal: goal node (Node type)
        - obstacles: list of obstacles
        - success_distance_threshold: distance threshold for a node to be considered close to goal (float)
    Return:
        - nodes: a list of nodes explored by RRT
    """
    nodes = [x_init]
    num_iter = 0
    while get_node_dist(nodes[-1], x_goal) > success_distance_threshold:
        x_rand = sample_free(x_goal.x, x_goal.y) # sample random point within map
        nearest_idx = nearest(x_rand, nodes) # get the index of nearest node from x_rand
        x_nearest = nodes[nearest_idx] # get nearest node from x_rand
        x_new = steer(x_nearest, x_rand) # steer towards x_rand from x_nearest and get a new node
        if obstacle_free(x_nearest, x_new, obstacles): # check if we can reach x_new from x_nearest
            x_new.parent = nearest_idx
            nodes.append(x_new)
        num_iter += 1
    return nodes

Let's now run RRT to find a feasible path!

In [ ]:
x_init = Node(cur_pos[0], cur_pos[1])
x_goal = Node(goal_pos[0], goal_pos[1])
nodes = rrt(x_init, x_goal, obstacles)
plt.figure()
plot_map(obstacles, cur_pos, goal_pos)
for node in nodes[1:]:
    x, y = node.x, node.y
    plt.gca().add_patch(patches.Circle((x, y), radius=0.1, color='red'))
    display.display(plt.gcf())
    display.clear_output(wait=True)
    time.sleep(0.01)
display.clear_output(wait=True)
print('Number of iterations: ', len(nodes)-1)

Now, we need to retrieve the path that will take the robot from its initial position to the goal. We do this by going backwards from the final node since we know the parent node of each node in our graph.

In [ ]:
def get_path(nodes):
    end_node = nodes[-1]
    path = [end_node]
    while(True):
        parent_node_idx = path[-1].parent
        path.append(nodes[parent_node_idx])
        if parent_node_idx == 0:
            break
    return path

In [ ]:
path = get_path(nodes)
plt.figure()
plot_map(obstacles, cur_pos, goal_pos)
for node in path:
    x, y = node.x, node.y
    plt.gca().add_patch(patches.Circle((x, y), radius=0.1, color='red'))